# Подготовка данных

## Объединяем наши данные: Отзывус, Тьютортоп, Сравнятор

### Отзывус

In [ ]:
import pandas as pd
import os

In [ ]:
# Загрузка данных
otzyvus_df = pd.read_csv("sources/otzyvus.csv", sep=";", index_col=False, low_memory=False)
# Оставляем только текст и оценки
otzyvus_df = otzyvus_df[['text', 'score_usefulness', 'score_complexity', 'score_interesting']]
# Дропаем строки, где текст отстутствует
otzyvus_df = otzyvus_df.dropna()
# Считаем рейтинг как среднее арифметическое оценок
otzyvus_df['rating'] = (otzyvus_df['score_usefulness'] + otzyvus_df['score_interesting']) / 2
# Оставляем только текст и рейтинг
otzyvus_df = otzyvus_df[['text', 'rating']]

/tmp/ipykernel_17170/3688423411.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  otzyvus_df = pd.read_csv("sources/otzyvus.csv", sep=";", index_col=False, low_memory=False)


### Сравнятор

In [ ]:
# Загружаем данные
sravnyator_df = pd.read_csv("sources/feedbacks_sravnyator_new.csv", sep=",", index_col=False, low_memory=False)
# Объединяем текст из разных колонок в один
sravnyator_df['text'] = sravnyator_df['dignities'] + " " + sravnyator_df['disadvantages'] + " " + sravnyator_df['impressions']
# Оставляем только текст и рейтинг
sravnyator_df = sravnyator_df[['text', 'rating']]

### Тьютортоп

In [ ]:
# Загружаем данные
tutortop_df = pd.read_csv("sources/feedback_tutortop.csv", sep=",", index_col=False, low_memory=False)
# Оставляем только текст и рейтинг
tutortop_df = tutortop_df[['text', 'rating']]

### Объединение

In [ ]:
sravnyator_df["rubrics"] = "образование_онлайн_курсы"
tutortop_df["rubrics"] = "образование_онлайн_курсы"
otzyvus_df["rubrics"] = "образование_онлайн_курсы;образование_отзывус"

courses_df = pd.concat([otzyvus_df, sravnyator_df, tutortop_df], ignore_index=True)

## Датасет Яндекса

In [ ]:
import time

data = {
    "address": [],
    "name_ru": [],
    "rubrics": [],
    "rating": [],
    "text": [],
}

with open('sources/geo-reviews-dataset-2023.tskv', 'r') as file:
    for line in file:
        tokens = line.split(u"\t")
        flags = {
            "address": False,
            "name_ru": False,
            "rubrics": False,
            "rating": False,
            "text": False,
        }
        for token in tokens:
            key_value = token.split(u"=")
            data[key_value[0]].append(key_value[1])
            flags[key_value[0]] = True
        for flag, value in flags.items():
            if not value:
                data[flag].append(None)

In [ ]:
# Оставляем только текст, рейтинг и рубрики
others_df = pd.DataFrame(data)[['text', 'rating', 'rubrics']]
# "\n" -> " "
others_df['text'] = others_df['text'].str.replace('\\n', ' ')

Наглядно

In [ ]:
courses_df

,text,rating,rubrics
0,"Был скучноват. Много практики, решение задач п...",3.0,образование_онлайн_курсы;образование_отзывус
1,Первые пол семестра - упражнения всякие сценич...,5.0,образование_онлайн_курсы;образование_отзывус
2,"Чисто основы основ 3д моделинга, для тех кто в...",4.0,образование_онлайн_курсы;образование_отзывус
3,"Электив просто песня, две презентации за весь ...",5.0,образование_онлайн_курсы;образование_отзывус
4,Прекрасный электив помогающий разобраться в се...,5.0,образование_онлайн_курсы;образование_отзывус
...,...,...,...
10354,Достижениями мне еще рано хвастаться. Только д...,2.0,образование_онлайн_курсы
10355,"Прошел уже большую часть курса, брал 2 раза ме...",5.0,образование_онлайн_курсы
10356,Начал обучение в этом приложении и это полный ...,1.0,образование_онлайн_курсы
10357,"Ребята, всем кто хочет практиковаться - в мире...",1.0,образование_онлайн_курсы


In [ ]:
others_df

,text,rating,rubrics
0,Московский квартал 2. Шумно : летом по ночам д...,3.,Жилой комплекс
1,"Замечательная сеть магазинов в общем, хороший ...",5.,Магазин продуктов;Продукты глубокой заморозки;...
2,"Не знаю смутят ли кого-то данные правила, но я...",1.,Фитнес-клуб
3,Хорошие условия аренды. Дружелюбный персонал....,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг
4,Топ мастер Ангелина топ во всех смыслах ) Немн...,5.,"Салон красоты;Визажисты, стилисты;Салон бровей..."
...,...,...,...
499995,"Охрана кривая но добрая, двери не закрываются ...",4.,Железнодорожная станция
499996,По сравнению со многими современными платформа...,4.,Железнодорожная станция
499997,"Приятная атмосфера, прекрасное вино, волшебная...",5.,"Бар, паб"
499998,Был с семьёй 13.06.23 Отличное место. Рекоменд...,5.,Достопримечательность


## Объединяем наш датасет и датасет Яндекса

In [ ]:
# Объединяем датасеты
all_df = pd.concat([courses_df, others_df], ignore_index=True)

In [ ]:
all_df

,text,rating,rubrics
0,"Был скучноват. Много практики, решение задач п...",3.0,образование_онлайн_курсы;образование_отзывус
1,Первые пол семестра - упражнения всякие сценич...,5.0,образование_онлайн_курсы;образование_отзывус
2,"Чисто основы основ 3д моделинга, для тех кто в...",4.0,образование_онлайн_курсы;образование_отзывус
3,"Электив просто песня, две презентации за весь ...",5.0,образование_онлайн_курсы;образование_отзывус
4,Прекрасный электив помогающий разобраться в се...,5.0,образование_онлайн_курсы;образование_отзывус
...,...,...,...
510354,"Охрана кривая но добрая, двери не закрываются ...",4.,Железнодорожная станция
510355,По сравнению со многими современными платформа...,4.,Железнодорожная станция
510356,"Приятная атмосфера, прекрасное вино, волшебная...",5.,"Бар, паб"
510357,Был с семьёй 13.06.23 Отличное место. Рекоменд...,5.,Достопримечательность


# Экспорт данных

In [ ]:
all_df.to_csv("dest/all_dataset.csv", sep='\t', encoding="utf-8")

In [ ]:
courses_df.to_csv("dest/courses_dataset.csv", sep='\t', encoding="utf-8")